In [1]:
# Install PuLP (if not already installed)
!pip install pulp
!apt-get install -y glpk-utils

# Intall and Import Package/Library
import pulp
from pulp import GLPK,LpMinimize
import pandas as pd
import numpy as np

data = pd.read_excel('/content/final dataset 1.xlsx' )
data = data.fillna(0)

# Nutritional constraints
nutrient_constraints = {
    "Energy (kcal) (kcal)": (2500, 1.1*2500),
    "Protein (g)": (55.5, None),
    "Fat (g)": (None, 97),
    "Satd FA /100g fd (g)": (None, 31),
    "Poly FA /100g food (g)": (18*0.9, 18*1.1),
    "Mono FA /100g food (g)": (36*0.9, 36*1.1),
    "Carbohydrate (g)": (333, None),
    "Free sugars (g)": (None, 33),
    "Salt(per 100gram)": (None, 6),

    "AOAC fibre (g)": (30, 70),
    #Dietary fibre < 70 https://ods.od.nih.gov/HealthInformation/nutrientrecommendations.aspx#dri

    #Upper limit source https://www.ncbi.nlm.nih.gov/books/NBK56068/table/summarytables.t7/?report=objectonly
    "Retinol (µg)": (700, 3000),
    "Thiamin (mg)": (1, None),
    "Riboflavin (mg)": (1.3, None),

    "Niacin equivalent (mg)": (16.5, 35),

    "Vitamin B6 (mg)": (1.4, 100),
    "Vitamin B12 (µg)": (1.5, None),
    "Folate (µg)": (200, 1000),
    "Vitamin C (mg)": (40, 2000),
    "Vitamin D (µg)": (10, 100),

    #https://www.ncbi.nlm.nih.gov/books/NBK545442/table/appJ_tab9/?report=objectonly
    "Iron (mg)": (8.7, 45),
    "Calcium (mg)": (700, 2500),
    "Magnesium (mg)": (300, 350),
    "Potassium (mg)": (3500, None),
    "Zinc (mg)": (9.5, 40),
    "Copper (mg)": (1.2, 10),
    "Iodine (µg)": (140, 1100),
    "Selenium (µg)": (75, 400),
    "Phosphorus (mg)": (550, 4000),

    "Chloride (mg)": (2500, 3600),

    #https://www.ncbi.nlm.nih.gov/books/NBK545442/table/appJ_tab3/?report=objectonly use usa standard
    "Sodium (mg)": (1500, None)
}
# Define fruits and vegetables group manually
fruits_and_vegetables = {
    "Apples, eating, raw, flesh and skin, weighed with core",
    "Avocado, Fuerte, flesh only, weighed with skin and stone",
    "Bananas, raw, flesh only, weighed with skin",
    "Blackberries, raw",
    "Blueberries",
    "Cherries, flesh and skin, raw",
    "Mangoes, ripe, flesh only, raw, weighed with skin and stone",
    "Oranges, flesh only, weighed with peel and pips",
    "Strawberries, raw",
    "Beans, cannellini, canned, re-heated, drained",
    "Beetroot, raw",
    "Broccoli, green, raw",
    "Cabbage, average, raw",
    "Carrots, old, raw",
    "Cauliflower, raw",
    "Garlic, raw",
    "Mushrooms, white, raw",
    "Onions, raw",
    "Peas, raw",
    "Spinach, mature, raw",
    "Tomatoes, standard, raw"
}

# Define red meat group manually
red_meat = {
    "Bacon rashers, back, raw",
    "Beef, flank, raw, lean",
    "Lamb, average, raw, lean and fat"
}

# Define staple foods (main sources of carbohydrates)
staple_foods = {
    "Bread, brown, toasted",
    "Rice, white, long grain, raw",
    "Potatoes, old, raw, flesh only",
    "Pasta, wholewheat, spaghetti, dried, raw"
}

Food_Name = data['Food Name']

# Define Environment & Direction of Optimization
prob = pulp.LpProblem("Diet_Cost_Minimization", LpMinimize)

# Define Decision Variables
food_vars = {row['Food Name']: pulp.LpVariable(row['Food Name'], lowBound=0, cat='Continuous') for _, row in data.iterrows()}

# Add Objective Function to the Environment
prob += pulp.lpSum([food_vars[row['Food Name']] * row['Price(Pound) per 100g'] for _, row in data.iterrows()]), "Total Cost"

# Build the matrix used for modeling
Constraint_matrix = []
Nutrition_matrix = []
for nutrient, (min_val, max_val) in nutrient_constraints.items():
  if min_val is not None:
      Constraint_matrix.append(-min_val)
      Nutrition_matrix.append(-data[nutrient].values)
  if max_val is not None:
      Constraint_matrix.append(max_val)
      Nutrition_matrix.append(data[nutrient].values)

Nutrition_matrix_np = np.array(Nutrition_matrix)
values = list(food_vars.values())

# Add Constraints to the Environment
for i in range(Nutrition_matrix_np.shape[0]):
    prob += pulp.lpSum([Nutrition_matrix_np[i, j] * values[j] for j in range(Nutrition_matrix_np.shape[1])]) <= Constraint_matrix[i]

# At least one-third of diet from fruits and vegetables
fruit_veg_items = data[data['Food Name'].isin(fruits_and_vegetables)]
prob += pulp.lpSum([food_vars[row['Food Name']] for _, row in fruit_veg_items.iterrows()]) >= 1/3 * pulp.lpSum([food_vars[row['Food Name']] for _, row in data.iterrows()]), "Fruit_Veg_Proportion"

# Define a dictionary to store binary variables indicating whether the food weight exceeds 80g
binary_vars = {row['Food Name']: pulp.LpVariable(f"Over_80g_{row['Food Name']}", cat='Binary') for _, row in fruit_veg_items.iterrows()}

# Add the constraint: Ensure at least 5 kinds of fruits and vegetables have a weight greater than 80g
for _, row in fruit_veg_items.iterrows():
    prob += food_vars[row['Food Name']] >= 0.8 * binary_vars[row['Food Name']], f"Weight_Over_80g_{row['Food Name']}"

# Add the constraint that at least 5 kinds of fruits/vegetables exceed 80g in weight
prob += pulp.lpSum([binary_vars[row['Food Name']] for _, row in fruit_veg_items.iterrows()]) >= 5, "Min_Fruit_Veg_Over_80g"

# Red meat limit (less than 70g)
red_meat_items = data[data['Food Name'].isin(red_meat)]
prob += pulp.lpSum([food_vars[row['Food Name']] for _, row in red_meat_items.iterrows()]) <= 0.7, "Max_Red_Meat"

# Ensure staple foods occupy at least one-third of the diet
staple_food_items = data[data['Food Name'].isin(staple_foods)]
prob += pulp.lpSum([food_vars[row['Food Name']] for _, row in staple_food_items.iterrows()]) >= 1/3 * pulp.lpSum([food_vars[row['Food Name']] for _, row in data.iterrows()]), "Staple_Food_Proportion"

# Solve the Problem
prob.solve(GLPK(msg=True))

# Print the status of the solution
print("\n\nModel Status: {}\n".format(pulp.LpStatus[prob.status]))

# Display optimal decision variables (amounts in grams) and reduced cost per variable
for v in prob.variables():
    # Check if the variable is a binary variable (indicating the 80g constraint) and skip it
    if not v.name.startswith ("Over_80g_"):
        continue
    if v.varValue is not None and v.varValue > 0:
        print(f"{v.name} = {v.varValue} ")





   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 42.1 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 49 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsuitesparseconfig5 amd64 1:5.10.1+dfsg-4build1 [10.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libamd2 amd64 1:5.10.1+dfsg-4build1 [21.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libcolamd2 amd64 1:5.10.1+dfsg-4build1 [18.0 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libglpk40 amd64 5.0